### Determining the optimal number of hidden layers and neurons for an Artificial Neural Network (ANN)
This can be challenging and often requires experimentation. However, there are some guidelines and methods that can help you in making an informed decision:
- Start Simple: Begin with a simple architecture and gradually increase complexity if needed
- Grid Search/Random Search: Use grid search or random search to try different architectures
- Cross-Validation: Use cross-validation to evaluate the performance of different architectures
- Heuistics and Rules of Thumb: Some heuristics and empirical rules can provide starting points, such as :
    - The number of neurons in the hidden layer should be between the size of the input layer and the size of output layer
    - A common practice is to start with 1-2 hidden layers

for performing hyperparameter tuning, there is one important function "KerasClassifier" which will help to find out the best model and for this we need to download the library called "scikeras"

In [ ]:
import pandas as pd 
